# LoopTree Tutorial

LoopTree is a model to evaluate the latency and energy of a fused-layer dataflow accelerator.

To model energy and latency, a workload, architecture, and mapping have to be specified. First, we discuss how these are specified. Then, we show how to run the LoopTree model.

## Specifying Architecture, Workload, and Mapping

For the LoopTree model to estimate energy and latency, the user must specify an architecture, workload, and mapping. Below, we discuss how to specify each of these inputs.

### Architecture
In LoopTree, the architecture of an accelerator is abstracted as buffers that use the Buffet semantics and computation units, following the [Timeloop v4 specification](https://timeloop.csail.mit.edu/v4).

An example architecture that we will use here is displayed below.

In [1]:
from util import *
from pprint import pp
show_config('architecture.yaml')

architecture:
  version: 0.4
  nodes:

  - !Component
    name: MainMemory
    class: DRAM
    attributes: {width: 256, block_size: 32, word_bits: 8, datawidth: 8}
    required_actions: ['read', 'write']



  - !Component
    name: IntermediateBuffer
    class: SRAM
    attributes:
      depth: 8192
      width: 64
      block_size: 32
      word_bits: 8
      datawidth: 8
  
  # - !Container
  #   name: system_arch
  #   attributes:
  #     # Top-level attributes inherited by all components unless overridden
  #     technology: "45nm"
  #     global_cycle_seconds: 1e-9
  #     datawidth: 16
    

  - !Component
    name: DRAM # offchip DRAM is the source of all datatypes
    class: DRAM # assume DRAM is large enough to store all the data, so no depth specification needed
    attributes:
      width: 64 # width in bits
      datawidth: 16
    required_actions: ['read', 'write']

  # - !Container
  #   name: chip

  # - !Container
  #   name: NNengine
  #   spatial: { meshX: 16, meshY: 

### Workloads

DNN workloads in LoopTree are abstracted as a set of Einsums, each representing a layer.

Each Einsum is specified as a dictionary. Each Einsum reads/writes to a number of tensors (referred to as `data_spaces` in the specification), and each tensor is either an input to the operation or an output (indicated by a key "read_write" with value `True`). Moreover, a "projection" from the Einsum to each tensor, which describes the index of the element in the data that an operation in the Einsum accesses, must be specified. Finally, we specify must specify the *shape* of the Einsum, which is the bounds of its dimensions (also referred to as "ranks").

An example comprising two fully-connected layers are shown below.

In [2]:
# show_config('two_fc.workload.yaml')

In this workload specification, we specify two Einsums Fc1 and Fc2. Each Einsum has three dimensions. Fc1 has dimensions P1, M1, and C1; Fc2 has dimensions P2, M2, and C2. Finally, we specify the shape of the Einsums as bounds for each Einsum dimension (rank).

### Mapping

The LoopTree mapping is a tree-structure that contains nodes of the types described below.
- **Loops**: a loop node specifies a rank in the Einsum that is partitioned and the shape of the tiles that results from the partitioning. If the loop is a "temporal" loop, then the tiles are scheduled to be processed one at a time. If the loop is a "spatial" loop, then the tiles are scheduled to parallel hardware units.
- **Branching point**: nodes above a branching point (*i.e.*, ancestor nodes) describe inter-Einsum mapping, which is applied to all Einsums under that branching point. Nodes underneath the branching point (*i.e.*, within each branch) pertain only to the Einsum of that branch.
- **Storage**: a storage node specifies the tiles of tensors to retain and which hardware level retains the tiles.
- **Compute**: a compute node specifies the hardware level used to compute operations of an Einsum. This node has to be a leaf node, and it also denotes the Einsum that a particular branch pertains to.

First, we discuss a layer-by-layer mapping example.

In [3]:
# show_config('layer-by-layer.mapping.yaml')

Explanation:
- (1.a) The storage node specifies that the tensors specified by the `dspace` key will be retained in target 0, which we will bind to MainMemory.
        Note that MainMemory retains Fmap2.
- (1.b) The sequential node specifies that the following branches (one for Fc1 and one for Fc2, as we will see shortly) are processed sequentially.
- (1.c) Filter1 is retained in GlobalBuffer
- (1.d) The rank P1 (which is a rank of the Fc1 Einsum) is partitioned into tiles with shape 1 and we will iterate over the tiles one at a time (temporal iteration).
- (1.e) Tiles of Fmap1 and Fmap2 are retained in GlobalBuffer. We retain tiles becuase the P1 rank has been partitioned.
- (1.f) Operations of Fc1 are processed in the MACC unit. Moreover, this node specifies that this branch is relevant to Fc2.
- (1.g) This and the following nodes specify how Fc2 is mapped.

In [4]:
# show_config('fused.mapping.yaml')

Explanation:
- (2.a) Similar to (1.a), but note that Fmap2 is no longer retained in MainMemory.
- (2.b) In this mapping, the $P2$ rank is partitioned to create tiles that are rows of feature maps Fmap1, Fmap2, and Fmap3. The tile shape attribute of the loop node implies that the tiles of Fmap3 have shape 1 in the $P2$ rank. LoopTree infers the shape of the tiles of Fmap2 and Fmap1 based on what is required as inputs to compute the specified tile of Fmap3. In this case, tiles that each contains one row of Fmap2 is required to compute tiles of Fmap3, and tiles that each contains one row of Fmap1 is required to compute tiles of Fmap2 in turn.
- (2.c) The tiles of Fmap1, Fmap2, and Fmap3 are retained in GlobalBuffer.
- (2.d) Similar to before, Fc1 and Fc2 are processed sequentially. However, as we have put node (2.b) above this sequential node, *tiles* of Fc1 and Fc2 are processed sequentially in alternating fashion: a tile of Fc1 produces a tile of Fmap2, the tile of Fmap2 is consumed by a tile of Fc2, another tile of Fc1 is produced, and so on.

## Running the Model

Running with the four matrix multiplications in a row (Test 1)

In [25]:
from pytimeloop.looptree.run import run_looptree
from util import *
from pprint import pp
# show_config('architecture.yaml')
# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'IntermediateBuffer',
    # 2: 'LocalBuffer',
    # 3: 'input_reg',
    # 4: 'weight_reg',
    # 5: 'output_reg',
    3: 'MACC'
}

stats = run_looptree(
    CONFIG_DIR,
    # ['architecture.yaml', 'two_fc.workload.yaml', 'layer-by-layer.mapping.yaml'],
    ['architecture.yaml', 'matrix_multiplication_workload.yaml','matrix_mapping.yaml'],
    TMP_DIR,
    bindings,
    call_accelergy=True
)

In [23]:
print('Latency:', stats.latency)

Latency: 8388608


and also energy, which is computed by calculating the number of actions to each hardware component and multiplying that with the energy per action estimated using the Accelergy tool:

In [24]:
print('Energy:')
pp(stats.energy)

Energy:
{('MainMemory', 'read'): 167772160.0,
 ('IntermediateBuffer', 'read'): 3054177943.552,
 ('IntermediateBuffer', 'write'): 974974713.8560001,
 ('MainMemory', 'write'): 67108864.0,
 ('MACC', 'compute'): 7088373.76}


In [169]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 KB 1.6 MB/s eta 0:00:0000:0100:01


In [63]:
### with CONV files (pipeline)
from util import *
from pprint import pp
from pytimeloop.looptree.run import run_looptree
import yaml

# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'IntermediateBuffer',
    # 2: 'LocalBuffer',
    # 3: 'input_reg',
    # 4: 'weight_reg',
    # 5: 'output_reg',
    3: 'MACC',
    4: 'input_reg',
    5: 'weights_reg',
    6: 'output_reg',
    # 7: 'PE'
}

#yaml configs

class Component(dict): pass
class Container(dict): pass

# --- YAML Constructors (load from file) ---
def component_constructor(loader, node):
    return Component(loader.construct_mapping(node))

def container_constructor(loader, node):
    return Container(loader.construct_mapping(node))

# --- YAML Representers (dump to file) ---
def component_representer(dumper, data):
    return dumper.represent_mapping('!Component', data)

def container_representer(dumper, data):
    return dumper.represent_mapping('!Container', data)

def quoted_str_presenter(dumper, data):
    return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='"')

yaml.add_representer(str, quoted_str_presenter)

# Register constructors
yaml.add_constructor('!Component', component_constructor, Loader=yaml.FullLoader)
yaml.add_constructor('!Container', container_constructor, Loader=yaml.FullLoader)

# Register representers
yaml.add_representer(Component, component_representer)
yaml.add_representer(Container, container_representer)


architecture_file = None
conv_mapping_file = None

# --- Load YAML ---
with open('../configs/architecture.yaml', 'r') as file:
    architecture_file = yaml.load(file, Loader=yaml.FullLoader)

with open('../configs/conv_mapping.yaml', 'r') as file:
    conv_mapping_file = yaml.load(file, Loader=yaml.FullLoader)

# get the parts of dictionary that define whether we are doing pipelined
# and tile_shape of each rank
for key in conv_mapping_file["mapping"]["nodes"]:
    if key['type'] == 'pipeline' or key['type'] == 'sequential':
        model = key
        pipelined = key['type']
        branches = key['branches']
        break
print(model)
# print(branches[1][0])
# get PE from dictionary
for key in architecture_file["architecture"]["nodes"]:
    if key['name'] == 'PE':
        PE = key
        break
# print(PE)

# pipelined = True

# model['type'] = 'pipeline' if pipelined else 'sequential'

# C2 = conv_mapping_file["mapping"]["nodes"][1]['tile_shape'], can be whatever?
# C = branches[0][1], pipelined ? tile_shape = 128/ (0.5 * PE_SIZE**2) : 128/ PE_SIZE**2 
# M2 branches[1][0], pipelined ? tile_shape = 128/ (0.5 * PE_SIZE**2) : 128/ PE_SIZE**2

# print(conv_mapping_file["mapping"]["nodes"][1]['tile_shape'])
# print(branches[1][0])

numNNEngines = 4
final_string = "PE_SIZE, latency, input_r, weights_r, output_r, dram_r, intbuffer_r, input_w, weights_w, output_w, dram_w, intbuffer_w, output_compute\n"
for i in range(1, 17):
    numPEs = i
    PE['spatial']['meshX'] = numPEs
    PE['spatial']['meshY'] = numPEs
    # PE['attributes']['meshX'] = numPEs
    # PE['attributes']['meshY'] = numPEs
    
    c2_tile_shape= 32
    c_tile_shape = tile_shape = int(128/ (0.5 * numPEs**2) if pipelined else 128/ numPEs**2)
    m2_tile_shape = tile_shape = int(128/ (0.5 * numPEs**2) if pipelined else 128/ numPEs**2)
    # print(c_tile_shape, m2_tile_shape)
    
    conv_mapping_file["mapping"]["nodes"][1]['tile_shape'] = c2_tile_shape
    branches[0][1]['tile_shape'] = c_tile_shape
    branches[1][0]['tile_shape'] = m2_tile_shape
    # print(conv_mapping_file)
    
    with open('../configs/architecture.yaml', 'w') as file:
        yaml.dump(architecture_file, file, default_flow_style=False)
    
    with open('../configs/conv_mapping.yaml', 'w') as file:
        yaml.dump(conv_mapping_file, file, default_flow_style=False)
    
    
    stats = run_looptree(
        CONFIG_DIR,
        # ['architecture.yaml', 'two_fc.workload.yaml', 'layer-by-layer.mapping.yaml'],
        ['architecture.yaml', 'conv_workload.yaml','conv_mapping.yaml'],
        TMP_DIR,
        bindings,
        call_accelergy=True
    )
    en = stats.energy
    final_string += f"{numPEs},{stats.latency/1e9},{en[('input_reg', 'read')]},{en[('weights_reg', 'read')]},{en[('output_reg', 'read')]},\
{en[('MainMemory', 'read')]},{en[('IntermediateBuffer', 'read')]},{en[('input_reg', 'write')]},{en[('weights_reg', 'write')]},{en[('output_reg', 'write')]},\
{en[('MainMemory', 'write')]},{en[('IntermediateBuffer', 'write')]},{en[('output_reg', 'compute')]}\n"

import csv

# string_data = "Name,Age,City\nJohn,30,New York\nAlice,25,London\nBob,35,Paris"

with open('peTestPipelineConv.csv', 'w', newline='\n') as csvfile:
    writer = csv.writer(csvfile)
    for row in final_string.splitlines():
        writer.writerow(row.split(','))

print(final_string)

{'branches': [[{'rank': 'M', 'tile_shape': 1, 'type': 'spatial'}, {'rank': 'C', 'tile_shape': 256, 'type': 'spatial'}, {'dspace': ['Input'], 'target': 4, 'type': 'storage'}, {'dspace': ['Filter_1'], 'target': 5, 'type': 'storage'}, {'dspace': ['Output'], 'target': 6, 'type': 'storage'}, {'rank': 'Q', 'tile_shape': 1, 'type': 'temporal'}, {'rank': 'P', 'tile_shape': 1, 'type': 'temporal'}, {'rank': 'C', 'tile_shape': 1, 'type': 'temporal'}, {'rank': 'M', 'tile_shape': 1, 'type': 'temporal'}, {'rank': 'R', 'tile_shape': 1, 'type': 'temporal'}, {'rank': 'S', 'tile_shape': 1, 'type': 'temporal'}, {'rank': 'N', 'tile_shape': 1, 'type': 'temporal'}, {'einsum': 'Conv1', 'target': 3, 'type': 'compute'}], [{'rank': 'M2', 'tile_shape': 256, 'type': 'spatial'}, {'rank': 'C2', 'tile_shape': 1, 'type': 'spatial'}, {'dspace': ['Output'], 'target': 4, 'type': 'storage'}, {'dspace': ['Filter_2'], 'target': 5, 'type': 'storage'}, {'dspace': ['Output2'], 'target': 6, 'type': 'storage'}, {'rank': 'M2', '

In [62]:
### with CONV files (sequential)
from util import *
from pprint import pp
from pytimeloop.looptree.run import run_looptree
import yaml

# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'IntermediateBuffer',
    # 2: 'LocalBuffer',
    # 3: 'input_reg',
    # 4: 'weight_reg',
    # 5: 'output_reg',
    3: 'MACC',
    4: 'input_reg',
    5: 'weights_reg',
    6: 'output_reg',
    # 7: 'PE'
}

#yaml configs

class Component(dict): pass
class Container(dict): pass

# --- YAML Constructors (load from file) ---
def component_constructor(loader, node):
    return Component(loader.construct_mapping(node))

def container_constructor(loader, node):
    return Container(loader.construct_mapping(node))

# --- YAML Representers (dump to file) ---
def component_representer(dumper, data):
    return dumper.represent_mapping('!Component', data)

def container_representer(dumper, data):
    return dumper.represent_mapping('!Container', data)

def quoted_str_presenter(dumper, data):
    return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='"')

yaml.add_representer(str, quoted_str_presenter)

# Register constructors
yaml.add_constructor('!Component', component_constructor, Loader=yaml.FullLoader)
yaml.add_constructor('!Container', container_constructor, Loader=yaml.FullLoader)

# Register representers
yaml.add_representer(Component, component_representer)
yaml.add_representer(Container, container_representer)


architecture_file = None
conv_mapping_file = None

# --- Load YAML ---
with open('../configs/architecture.yaml', 'r') as file:
    architecture_file = yaml.load(file, Loader=yaml.FullLoader)

with open('../configs/conv_mapping_sequential.yaml', 'r') as file:
    conv_mapping_file = yaml.load(file, Loader=yaml.FullLoader)

# # get the parts of dictionary that define whether we are doing pipelined
# # and tile_shape of each rank
for key in conv_mapping_file["mapping"]["nodes"]:
    if key['type'] == 'pipeline' or key['type'] == 'sequential':
        model = key
        pipelined = key['type']
        branches = key['branches']
        break

# print(f"{branches[1][1]=}, {branches[3][0]=}")
# # for key in branches:
#     # print(key)

# # print(model)
# print(f"{branches=}")
# # get PE from dictionary
for key in architecture_file["architecture"]["nodes"]:
    if key['name'] == 'PE':
        PE = key
        break
# # print(PE)

# # pipelined = True

# # model['type'] = 'pipeline' if pipelined else 'sequential'

# # C2 = conv_mapping_file["mapping"]["nodes"][1]['tile_shape'], can be whatever?
# # C = branches[0][1], pipelined ? tile_shape = 128/ (0.5 * PE_SIZE**2) : 128/ PE_SIZE**2 
# # M2 branches[1][0], pipelined ? tile_shape = 128/ (0.5 * PE_SIZE**2) : 128/ PE_SIZE**2

# # print(conv_mapping_file["mapping"]["nodes"][1]['tile_shape'])
# # print(branches[1][0])
# print(f"{branches=}")
# print(f"{branches[0][2]=}")
# print(f"{branches[1][1]=}")

numNNEngines = 4
final_string = "PE_SIZE, latency, input_r, weights_r, output_r, dram_r, intbuffer_r, input_w, weights_w, output_w, dram_w, intbuffer_w, output_compute\n"
for i in range(1, 17):
    numPEs = i
    PE['spatial']['meshX'] = numPEs
    PE['spatial']['meshY'] = numPEs
    PE['attributes']['meshX'] = numPEs
    PE['attributes']['meshY'] = numPEs
    
    c2_tile_shape= 32
    c_tile_shape = tile_shape = int(128/ (0.5 * numPEs**2) if pipelined else 128/ numPEs**2)
    m2_tile_shape = tile_shape = int(128/ (0.5 * numPEs**2) if pipelined else 128/ numPEs**2)
    # print(c_tile_shape, m2_tile_shape)
    
    conv_mapping_file["mapping"]["nodes"][1]['tile_shape'] = c2_tile_shape
    branches[0][2]['tile_shape'] = c_tile_shape
    branches[1][0]['tile_shape'] = m2_tile_shape
    # print(conv_mapping_file)
    
    with open('../configs/architecture.yaml', 'w') as file:
        yaml.dump(architecture_file, file, default_flow_style=False)
    
    with open('../configs/conv_mapping_sequential.yaml', 'w') as file:
        yaml.dump(conv_mapping_file, file, default_flow_style=False)
    
    
    stats = run_looptree(
        CONFIG_DIR,
        # ['architecture.yaml', 'two_fc.workload.yaml', 'layer-by-layer.mapping.yaml'],
        ['architecture.yaml', 'conv_workload.yaml','conv_mapping_sequential.yaml'],
        TMP_DIR,
        bindings,
        call_accelergy=True
    )
    en = stats.energy
    final_string += f"{numPEs},{stats.latency/1e9},{en[('input_reg', 'read')]},{en[('weights_reg', 'read')]},{en[('output_reg', 'read')]},\
    {en[('MainMemory', 'read')]},{en[('IntermediateBuffer', 'read')]},{en[('input_reg', 'write')]},{en[('weights_reg', 'write')]},{en[('output_reg', 'write')]},\
    {en[('MainMemory', 'write')]},{en[('IntermediateBuffer', 'write')]},{en[('output_reg', 'compute')]}\n"
# replace {0} with {en[('IntermediateBuffer', 'read')]} if bringing back intbuffer
# same for intbuffer write
import csv

string_data = "Name,Age,City\nJohn,30,New York\nAlice,25,London\nBob,35,Paris"

with open('peTestSequentialConv.csv', 'w', newline='\n') as csvfile:
    writer = csv.writer(csvfile)
    for row in final_string.splitlines():
        writer.writerow(row.split(','))

print(final_string)

PE_SIZE, latency, input_r, weights_r, output_r, dram_r, intbuffer_r, input_w, weights_w, output_w, dram_w, intbuffer_w, output_compute
1,35184.372088832,76493639670887.88,597606559928.8115,76493603195878.11,    140738562097152.0,129633107558.40001,2388579950.592,36475009.76128,76495955834007.72,    70368744177664.0,30897457856.512,76493639670887.88
2,17592.186044416,76493639670887.88,597606559928.8115,76493603195878.11,    140738562097152.0,291929172967.424,2460961161.216,36475009.76128,76495974071512.6,    70368744177664.0,61794915713.024,76493639670887.88
3,7696.581394432,76493639670887.88,597606559928.8115,76493603195878.11,    140738562097152.0,778817369194.496,2678104793.088,36475009.76128,76496028784027.23,    70368744177664.0,154487289282.56,76493639670887.88
4,4398.046511104,76493639670887.88,597606559928.8115,76493603195878.11,    140738562097152.0,1265705565421.568,2895248424.96,36475009.76128,76496083496541.88,    70368744177664.0,247179662852.096,76493639670887.88
5,2748.77

In [44]:
### with CONV files but for changing the channels, sequential
from util import *
from pprint import pp
from pytimeloop.looptree.run import run_looptree
import yaml

# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'IntermediateBuffer',
    # 2: 'LocalBuffer',
    # 3: 'input_reg',
    # 4: 'weight_reg',
    # 5: 'output_reg',
    3: 'MACC',
    4: 'input_reg',
    5: 'weights_reg',
    6: 'output_reg',
    # 7: 'PE'
}

workload_file = None

# --- Load YAML ---
with open('../configs/conv_workload.yaml', 'r') as file:
    workload_file = yaml.load(file, Loader=yaml.FullLoader)


# get the parts of dictionary that define whether we are doing pipelined
# and tile_shape of each rank



print(workload_file["problem"])

final_string = "channels, latency, input_r, weights_r, output_r, dram_r, intbuffer_r, input_w, weights_w, output_w, dram_w, intbuffer_w, output_compute\n"


# workload_file["problem"][

for i in range(3, 8):
    layer_one_input_channels = 2**i
    layer_two_input_channels = 2**i
    layer_two_output_channels = 2**i
    
    instance_string_one = f"0 <= N < 128 and 0 <= M < {layer_two_input_channels} and 0 <= P < 128 and 0 <= Q < 128 and 0 <= C < {layer_one_input_channels} and 0 <= R < 128 and 0 <= S < 128"
    
    instance_string_two = f"0 <= N2 < 128 and 0 <= M2 < {layer_two_output_channels} and 0 <= P2 < 128 and 0 <= Q2 < 128 and 0 <= C2 < {layer_two_input_channels} and 0 <= R2 < 128 and 0 <= S2 < 128"
    
    for key in workload_file["problem"]:
        name = key['shape']['name']
        # print(key)
        if name == 'Conv1':
            key['instance'] = instance_string_one
        else:
            key['instance'] = instance_string_two

    with open('../configs/conv_workload.yaml', 'w') as file:
        yaml.dump(workload_file, file, default_flow_style=False)

    stats = run_looptree(
        CONFIG_DIR,
        # ['architecture.yaml', 'two_fc.workload.yaml', 'layer-by-layer.mapping.yaml'],
        ['architecture.yaml', 'conv_workload.yaml','conv_mapping_sequential.yaml'],
        TMP_DIR,
        bindings,
        call_accelergy=True
    )
    en = stats.energy
    final_string += f"{2**i},{stats.latency/1e9},{en[('input_reg', 'read')]},{en[('weights_reg', 'read')]},{en[('output_reg', 'read')]},\
    {en[('MainMemory', 'read')]},{en[('IntermediateBuffer', 'read')]},{en[('input_reg', 'write')]},{en[('weights_reg', 'write')]},{en[('output_reg', 'write')]},\
    {en[('MainMemory', 'write')]},{en[('IntermediateBuffer', 'write')]},{en[('output_reg', 'compute')]}\n"

import csv

# string_data = "Name,Age,City\nJohn,30,New York\nAlice,25,London\nBob,35,Paris"

with open('channelTestSequentialConv.csv', 'w', newline='\n') as csvfile:
    writer = csv.writer(csvfile)
    for row in final_string.splitlines():
        writer.writerow(row.split(','))

print(final_string)

[{'instance': '0 <= N < 128 and 0 <= M < 128 and 0 <= P < 128 and 0 <= Q < 128 and 0 <= C < 128 and 0 <= R < 128 and 0 <= S < 128', 'shape': {'data_spaces': [{'dimensions': ['I_batches', 'I_Channels', 'I_Height', 'I_Width'], 'name': 'Input', 'projection': '[ N, C, P + R, Q + S]'}, {'dimensions': ['F_batches', 'F_channels', 'F_Height', 'F_Width'], 'name': 'Filter_1', 'projection': '[ M, C, R, S ]'}, {'dimensions': ['O_in_b', 'O_out_b', 'O_Height', 'O_Width'], 'name': 'Output', 'projection': '[ N, M, P, Q ]', 'read_write': True}], 'dimensions': ['C', 'M', 'R', 'S', 'N', 'P', 'Q'], 'name': 'Conv1'}}, {'instance': '0 <= N2 < 128 and 0 <= M2 < 128 and 0 <= P2 < 128 and 0 <= Q2 < 128 and 0 <= C2 < 128 and 0 <= R2 < 128 and 0 <= S2 < 128', 'shape': {'data_spaces': [{'dimensions': ['I_batches2', 'I_Channels2', 'I_Height2', 'I_Width2'], 'name': 'Output', 'projection': '[ N2, C2, P2 + R2, Q2 + S2]'}, {'dimensions': ['F_batches2', 'F_channels2', 'F_Height2', 'F_Width2'], 'name': 'Filter_2', 'proj

In [45]:
### with CONV files but for changing the channels, pipeline
from util import *
from pprint import pp
from pytimeloop.looptree.run import run_looptree
import yaml

# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'IntermediateBuffer',
    # 2: 'LocalBuffer',
    # 3: 'input_reg',
    # 4: 'weight_reg',
    # 5: 'output_reg',
    3: 'MACC',
    4: 'input_reg',
    5: 'weights_reg',
    6: 'output_reg',
    # 7: 'PE'
}

workload_file = None

# --- Load YAML ---
with open('../configs/conv_workload.yaml', 'r') as file:
    workload_file = yaml.load(file, Loader=yaml.FullLoader)


# get the parts of dictionary that define whether we are doing pipelined
# and tile_shape of each rank



print(workload_file["problem"])

final_string = "channels, latency, input_r, weights_r, output_r, dram_r, intbuffer_r, input_w, weights_w, output_w, dram_w, intbuffer_w, output_compute\n"


# workload_file["problem"][

for i in range(3, 8):
    layer_one_input_channels = 2**i
    layer_two_input_channels = 2**i
    layer_two_output_channels = 2**i
    
    instance_string_one = f"0 <= N < 128 and 0 <= M < {layer_two_input_channels} and 0 <= P < 128 and 0 <= Q < 128 and 0 <= C < {layer_one_input_channels} and 0 <= R < 128 and 0 <= S < 128"
    
    instance_string_two = f"0 <= N2 < 128 and 0 <= M2 < {layer_two_output_channels} and 0 <= P2 < 128 and 0 <= Q2 < 128 and 0 <= C2 < {layer_two_input_channels} and 0 <= R2 < 128 and 0 <= S2 < 128"
    
    for key in workload_file["problem"]:
        name = key['shape']['name']
        # print(key)
        if name == 'Conv1':
            key['instance'] = instance_string_one
        else:
            key['instance'] = instance_string_two

    with open('../configs/conv_workload.yaml', 'w') as file:
        yaml.dump(workload_file, file, default_flow_style=False)

    stats = run_looptree(
        CONFIG_DIR,
        # ['architecture.yaml', 'two_fc.workload.yaml', 'layer-by-layer.mapping.yaml'],
        ['architecture.yaml', 'conv_workload.yaml','conv_mapping.yaml'],
        TMP_DIR,
        bindings,
        call_accelergy=True
    )
    en = stats.energy
    final_string += f"{2**i},{stats.latency/1e9},{en[('input_reg', 'read')]},{en[('weights_reg', 'read')]},{en[('output_reg', 'read')]},\
    {en[('MainMemory', 'read')]},{en[('IntermediateBuffer', 'read')]},{en[('input_reg', 'write')]},{en[('weights_reg', 'write')]},{en[('output_reg', 'write')]},\
    {en[('MainMemory', 'write')]},{en[('IntermediateBuffer', 'write')]},{en[('output_reg', 'compute')]}\n"

import csv

# string_data = "Name,Age,City\nJohn,30,New York\nAlice,25,London\nBob,35,Paris"

with open('channelTestPipelineConv.csv', 'w', newline='\n') as csvfile:
    writer = csv.writer(csvfile)
    for row in final_string.splitlines():
        writer.writerow(row.split(','))

print(final_string)

[{'instance': '0 <= N < 128 and 0 <= M < 128 and 0 <= P < 128 and 0 <= Q < 128 and 0 <= C < 128 and 0 <= R < 128 and 0 <= S < 128', 'shape': {'data_spaces': [{'dimensions': ['I_batches', 'I_Channels', 'I_Height', 'I_Width'], 'name': 'Input', 'projection': '[ N, C, P + R, Q + S]'}, {'dimensions': ['F_batches', 'F_channels', 'F_Height', 'F_Width'], 'name': 'Filter_1', 'projection': '[ M, C, R, S ]'}, {'dimensions': ['O_in_b', 'O_out_b', 'O_Height', 'O_Width'], 'name': 'Output', 'projection': '[ N, M, P, Q ]', 'read_write': True}], 'dimensions': ['C', 'M', 'R', 'S', 'N', 'P', 'Q'], 'name': 'Conv1'}}, {'instance': '0 <= N2 < 128 and 0 <= M2 < 128 and 0 <= P2 < 128 and 0 <= Q2 < 128 and 0 <= C2 < 128 and 0 <= R2 < 128 and 0 <= S2 < 128', 'shape': {'data_spaces': [{'dimensions': ['I_batches2', 'I_Channels2', 'I_Height2', 'I_Width2'], 'name': 'Output', 'projection': '[ N2, C2, P2 + R2, Q2 + S2]'}, {'dimensions': ['F_batches2', 'F_channels2', 'F_Height2', 'F_Width2'], 'name': 'Filter_2', 'proj

In [273]:
### with NEW MATRIX files
from util import *
from pprint import pp
from pytimeloop.looptree.run import run_looptree
import yaml

# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'IntermediateBuffer',
    # 2: 'LocalBuffer',
    # 3: 'input_reg',
    # 4: 'weight_reg',
    # 5: 'output_reg',
    3: 'MACC',
    4: 'input_reg',
    5: 'weights_reg',
    6: 'output_reg',
    # 7: 'PE'
}

#yaml configs

class Component(dict): pass
class Container(dict): pass

# --- YAML Constructors (load from file) ---
def component_constructor(loader, node):
    return Component(loader.construct_mapping(node))

def container_constructor(loader, node):
    return Container(loader.construct_mapping(node))

# --- YAML Representers (dump to file) ---
def component_representer(dumper, data):
    return dumper.represent_mapping('!Component', data)

def container_representer(dumper, data):
    return dumper.represent_mapping('!Container', data)



# Register constructors
yaml.add_constructor('!Component', component_constructor, Loader=yaml.FullLoader)
yaml.add_constructor('!Container', container_constructor, Loader=yaml.FullLoader)

# Register representers
yaml.add_representer(Component, component_representer)
yaml.add_representer(Container, container_representer)


architecture_file = None
conv_mapping_file = None

# --- Load YAML ---
with open('../configs/architecture.yaml', 'r') as file:
    architecture_file = yaml.load(file, Loader=yaml.FullLoader)

with open('../configs/new_matrix_mapping.yaml', 'r') as file:
    conv_mapping_file = yaml.load(file, Loader=yaml.FullLoader)

# get the parts of dictionary that define whether we are doing pipelined
# and tile_shape of each rank
for key in conv_mapping_file["mapping"]["nodes"]:
    if key['type'] == 'pipeline' or key['type'] == 'sequential':
        model = key
        pipelined = key['type']
        branches = key['branches']
        break
print(model)
# print(branches[1][0])
# get PE from dictionary
for key in architecture_file["architecture"]["nodes"]:
    if key['name'] == 'PE':
        PE = key
        break
# print(PE)

# pipelined = True

# model['type'] = 'pipeline' if pipelined else 'sequential'

# C2 = conv_mapping_file["mapping"]["nodes"][1]['tile_shape'], can be whatever?
# C = branches[0][1], pipelined ? tile_shape = 128/ (0.5 * PE_SIZE**2) : 128/ PE_SIZE**2 
# M2 branches[1][0], pipelined ? tile_shape = 128/ (0.5 * PE_SIZE**2) : 128/ PE_SIZE**2

# print(conv_mapping_file["mapping"]["nodes"][1]['tile_shape'])
# print(branches[1][0])

numNNEngines = 4

#no more intbuffer because of matrix???
final_string = "PE_SIZE, latency, input_r, weights_r, output_r, dram_r, intbuffer_r, input_w, weights_w, output_w, dram_w, output_compute\n"
for i in range(1, 17):
    numPEs = i
    PE['spatial']['meshX'] = numPEs
    PE['spatial']['meshY'] = numPEs
    # PE['attributes']['meshX'] = numPEs
    # PE['attributes']['meshY'] = numPEs
    
    c2_tile_shape= 32
    c_tile_shape = tile_shape = int(128/ (0.5 * numPEs**2) if pipelined else 128/ numPEs**2)
    m2_tile_shape = tile_shape = int(128/ (0.5 * numPEs**2) if pipelined else 128/ numPEs**2)
    # print(c_tile_shape, m2_tile_shape)
    
    conv_mapping_file["mapping"]["nodes"][1]['tile_shape'] = c2_tile_shape
    branches[0][1]['tile_shape'] = c_tile_shape
    branches[1][0]['tile_shape'] = m2_tile_shape
    # print(conv_mapping_file)
    
    with open('../configs/architecture.yaml', 'w') as file:
        yaml.dump(architecture_file, file, default_flow_style=False)
    
    with open('../configs/new_matrix_mapping.yaml', 'w') as file:
        yaml.dump(conv_mapping_file, file, default_flow_style=False)
    
    
    stats = run_looptree(
        CONFIG_DIR,
        # ['architecture.yaml', 'two_fc.workload.yaml', 'layer-by-layer.mapping.yaml'],
        ['architecture.yaml', 'new_matrix_workload.yaml','new_matrix_mapping.yaml'],
        TMP_DIR,
        bindings,
        call_accelergy=True
    )
    en = stats.energy
    # print(en)
    # no more intbuffer because of matrix???
    final_string += f"{numPEs},{stats.latency/1e3},{en[('input_reg', 'read')]},{en[('weights_reg', 'read')]},{en[('output_reg', 'read')]},\
{en[('MainMemory', 'read')]},{en[('IntermediateBuffer', 'read')]},{en[('input_reg', 'write')]},{en[('weights_reg', 'write')]},{en[('output_reg', 'write')]},\
{en[('MainMemory', 'write')]},{en[('output_reg', 'compute')]}\n"

import csv

# string_data = "Name,Age,City\nJohn,30,New York\nAlice,25,London\nBob,35,Paris"

with open('peTestPipelineMatrix.csv', 'w', newline='\n') as csvfile:
    writer = csv.writer(csvfile)
    for row in final_string.splitlines():
        writer.writerow(row.split(','))

print(final_string)

{'branches': [[{'rank': 'M', 'tile_shape': 1, 'type': 'spatial'}, {'rank': 'C', 'tile_shape': 1, 'type': 'spatial'}, {'dspace': ['Input'], 'target': 4, 'type': 'storage'}, {'dspace': ['Filter_1'], 'target': 5, 'type': 'storage'}, {'dspace': ['Output'], 'target': 6, 'type': 'storage'}, {'rank': 'Q', 'tile_shape': 1, 'type': 'temporal'}, {'rank': 'P', 'tile_shape': 1, 'type': 'temporal'}, {'rank': 'C', 'tile_shape': 1, 'type': 'temporal'}, {'rank': 'M', 'tile_shape': 1, 'type': 'temporal'}, {'rank': 'R', 'tile_shape': 1, 'type': 'temporal'}, {'rank': 'S', 'tile_shape': 1, 'type': 'temporal'}, {'rank': 'N', 'tile_shape': 1, 'type': 'temporal'}, {'einsum': 'Conv1', 'target': 3, 'type': 'compute'}], [{'rank': 'M2', 'tile_shape': 1, 'type': 'spatial'}, {'rank': 'C2', 'tile_shape': 1, 'type': 'spatial'}, {'dspace': ['Output'], 'target': 4, 'type': 'storage'}, {'dspace': ['Filter_2'], 'target': 5, 'type': 'storage'}, {'dspace': ['Output2'], 'target': 6, 'type': 'storage'}, {'rank': 'M2', 'tile

KeyboardInterrupt: 

In [167]:
print('Latency:', stats.latency/1e9)
print('Energy:')
pp(stats.energy)

Latency: 549.755813888
Energy:
{('input_reg', 'read'): 76493639670887.88,
 ('weights_reg', 'read'): 597606559928.8115,
 ('output_reg', 'read'): 76493603195878.11,
 ('MainMemory', 'read'): 73100376932352.0,
 ('IntermediateBuffer', 'read'): 2187399452147.7122,
 ('output_reg', 'write'): 76497104796815.2,
 ('input_reg', 'write'): 6948596219.904,
 ('weights_reg', 'write'): 36475009.76128,
 ('IntermediateBuffer', 'write'): 1977437302816.768,
 ('MainMemory', 'write'): 70368744177664.0,
 ('output_reg', 'compute'): 76493639670887.88}


In [70]:
pp(stats)

LoopTreeStatistics(latency=687194767360,
                   energy={('input_reg', 'read'): 76493639670887.88,
                           ('weights_reg', 'read'): 597606559928.8115,
                           ('output_reg', 'read'): 76493603195878.11,
                           ('MainMemory', 'read'): 73100376932352.0,
                           ('IntermediateBuffer', 'read'): 1142184800927.7441,
                           ('output_reg', 'write'): 76496521196659.02,
                           ('input_reg', 'write'): 4632397479.936,
                           ('weights_reg', 'write'): 36475009.76128,
                           ('IntermediateBuffer', 'write'): 988718651408.384,
                           ('MainMemory', 'write'): 70368744177664.0,
                           ('output_reg', 'compute'): 76493639670887.88},
                   actions={('input_reg', 'read'): 1125899906842624,
                            ('weights_reg', 'read'): 8796093022208,
                            ('outpu

Now, we compare these results with the fused mapping:

In [1]:
from pytimeloop.looptree.run import run_looptree

# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'GlobalBuffer',
    2: 'MACC'
}

stats = run_looptree(
    CONFIG_DIR,
    ['architecture.yaml', 'two_fc.workload.yaml', 'fused.mapping.yaml'],
    TMP_DIR,
    bindings,
    call_accelergy=True
)
print('Latency:', stats.latency)
print('Energy:')
pp(stats.energy)

NameError: name 'CONFIG_DIR' is not defined

As we can see, the energy consumption due to MainMemory reads and writes have decreased significantly from fusion.

Here, we have not modeled the impact of limited MainMemory bandwidth; thus, because the MACC unit utilization is the same (100%) in both cases, the latency is the same. However, if the layer-by-layer latency is limited by MainMemory bandwidth, fusion will also decrease latency.

In [19]:
from pytimeloop.looptree.run import run_looptree

# As previously mentioned, bindings map levels specified in the mapping
# to the hardware units specified in the architecture spec.
bindings = {
    0: 'MainMemory',
    1: 'GlobalBuffer',
    2: 'MACC'
}

stats = run_looptree(
    CONFIG_DIR,
    ['architecture.yaml', 'two_fc.workload.yaml', 'fused-pipeline.mapping.yaml'],
    TMP_DIR,
    bindings,
    call_accelergy=True
)
print('Latency:', stats.latency)
print('Energy:')
pp(stats.energy)

Latency: 528384
Energy:
{('MainMemory', 'read'): 50331648.0,
 ('GlobalBuffer', 'read'): 382769037.31200004,
 ('GlobalBuffer', 'write'): 123521941.50400001,
 ('MainMemory', 'write'): 16777216.0,
 ('MACC', 'compute'): 886046.72}
